   ### Link to: [Preprocesing](preprocessing.ipynb)     [Input](input.ipynb)     [Processing](processing.ipynb)     [Postprocessing](postprocessing.ipynb)

        

# Processing

## Table of Contents
<div class="toc" style="margin-top: 1em;">
   <ul class="toc-item" id="toc-level0">
      <li><span><a href='#part 1' data-toc-modified-id="part 1"><span class="toc-item-num">1&nbsp;&nbsp;</span>part 1</a></span></li>
      <li><span><a href='#part 2' data-toc-modified-id="part 2"><span class="toc-item-num">2&nbsp;&nbsp;</span>part 2</a></span></li>
      <li><span><a href='#part 3' data-toc-modified-id="part 3"><span class="toc-item-num">3&nbsp;&nbsp;</span>part 3</a></span></li>
      <li><span><a href='#part 4' data-toc-modified-id="part 4"><span class="toc-item-num">4&nbsp;&nbsp;</span>part 4</a></span></li>
      <li><span><a href='#part 5' data-toc-modified-id="part 5"><span class="toc-item-num">5&nbsp;&nbsp;</span>part 5</a></span></li>
   </ul>
</div>




# Part 1
<a id='part 1'></a>

In [ ]:
library(rhdf5)
library(RCurl) #for merge list
library(xts)




SyntaxError: invalid syntax (<ipython-input-1-0d3618ef1590>, line 20)

# Part 2
<a id='part 2'></a>

# Part 3
<a id='part 3'></a>


# Part 4
<a id='part 4'></a>


In [ ]:
class ThreadManager:
    def __call__(self, num_rows=5):
        """Returns the num_rows most recent threads (Default: 5)"""
        # Update exec time if thread has yet to finish
        for ix in self.thread_df.index:
            if self.thread_df.loc[ix, 'finish_time'] == '':
                start_time = self.thread_df.loc[ix, 'start_time']
                time_elapsed = datetime.now() - start_time
                
                self.thread_df.loc[ix, 'exec_time'] = time_elapsed
        return self.thread_df.tail(num_rows).iloc[::-1]
    
    def __init__(self):
        df_cols = ['start_time', 'finish_time', 'exec_time',
                   'cell_text', 'comment', 'error_message']
        self.thread_df = pd.DataFrame(columns=df_cols)
        
    def _add_finish_times(self, thread_id):
        """Adds the finish time and exec time to thread_df.
        
        Returns: a tuple of finish time, exec time
        """
        
        # Set finish time
        finish_time = datetime.now()
        self.thread_df.loc[thread_id, 'finish_time'] = finish_time

        # Set execution time
        exec_time = finish_time - self.thread_df.loc[thread_id, 'start_time']
        self.thread_df.loc[thread_id, 'exec_time'] = exec_time
        
        return finish_time, exec_time
    
    def get_error_threads(self):
        """Returns a DataFrame of threads which threw an error."""
        return self.thread_df[self.thread_df['error_message'] != '']
    
    def get_finished_threads(self):
        """Returns a DataFrame of the finished threads."""
        return self.thread_df[self.thread_df['finish_time'] != '']
        
    def get_next_thread_id(self):
        """Returns an integer representing the ID for the next thread."""
        return self.thread_df.shape[0]

    def get_unfinished_threads(self):
        """Returns a DataFrame of the unfinished threads."""
        return self.thread_df[self.thread_df['finish_time'] == '']
    
    def add_thread(self, cell_text, comment='N/A', hide_output=False):
        """Adds a new background thread."""
        thread_id = self.get_next_thread_id()
        start_time = datetime.now()
        
        self.thread_df.loc[thread_id] = [start_time, '', '', cell_text, comment, '']
        if not hide_output:
            print 'Started Thread {} at {}.\nComment: {}'\
                .format(thread_id, start_time, comment)
        
    def finish_thread(self, thread_id, hide_output=False):
        """Completes a thread."""
        if self.thread_df.loc[thread_id, 'finish_time'] == '':
            finish_time, exec_time = self._add_finish_times(thread_id)
            
            # Print comment
            comment = str(self.thread_df.loc[thread_id, 'comment'])
            if not hide_output:
                print 'Finished Thread {} at {}.\nDone in {}.\nComment: {}'\
                    .format(thread_id, finish_time, exec_time, comment)
                        
        else:
            raise Exception('Cannot finish an already completed thread.')
        
    def raise_thread_error(self, thread_id, error_message):
        if self.thread_df.loc[thread_id, 'finish_time'] == '':
            exception_message = 'Exception: {}'.format(error_message)
            self._add_finish_times(thread_id)            
            self.thread_df.loc[thread_id, 'error_message'] = exception_message
        
thread_manager = ThreadManager()

# Part 5
<a id='part 5'></a>
These functions allow us to type write and run raw SQL a cell with the magic function at the top.

In [ ]:
@register_cell_magic
def readsql(line, cell):
    """
    Extract the code in the specific cell (should be valid SQL), and 
    execute it using the connection object to the backend  database.
    The resulting pandas DataFrame is rendered inline  below the cell
    using IPython.display. You'd use this for SELECT.
    
    Returns a DataFrame with the name specified in the magic function.
    If this is not specified, then the DataFrame is called _df. This
    also takes in an option "-h", followed by a number. This will show
    only the specified number of rows in the DataFrame.
    """
    
    # Use the global connection object defined above.
    global conn
    optlist, args = getopt.getopt(line.split(), 'ih:')
    optdict = dict(optlist) 
    # If '-h' tag is specified, set the number of rows to display
    if '-h' in optdict:
        head_num = int(optdict['-h'])
    
    # Do string formatting. If a PL/Python function
    # is being created, then it should not try and
    # format whatever is inside the function.
    split_cell = cell.split('$')
    if '-i' not in optdict:
        if len(split_cell) > 1:
            split_cell[0] = split_cell[0].format(**globals())
            split_cell[-1] = split_cell[-1].format(**globals())
            cell = '$'.join(split_cell)
        elif len(split_cell) == 1:
            cell = cell.format(**globals()) 
    
    # If there is more than one table name specified,
    # throw an exception.
    if len(args) > 1:
        raise Exception('More than one table name specified.')

    elif len(args) == 1:
        # If a table name is specified, store it as that
        table_name = args[0]
        globals()[table_name] = psql.read_sql(cell, conn)
        if '-h' in optdict:
            # If head_num is not 0, then display rows
            if head_num != 0:
                display(globals()[table_name].head(int(optdict['-h'])))
        else:
            display(globals()[table_name])

    else:
        # Otherwise, call it _df
        global _df
        _df = psql.read_sql(cell, conn)
        if '-h' in optdict:
            # If head_num is not 0, then display rows
            if head_num != 0:
                display(_df.head(head_num))
        else:
            display(_df)
            
    refresh_tables(conn)


@register_cell_magic
def execsql(line, cell):
    """
    Extract the code in the specific cell (should be valid SQL), and
    execute it using the connection object to the backend  database.
    You'd use this for CREATE/UPDATE/DELETE.
    """
    
    # Use the global connection object defined above.
    global conn
    optlist, args = getopt.getopt(line.split(), 'ih:')
    optdict = dict(optlist)
    
    # Do string formatting. If a PL/Python function
    # is being created, then it should not try and
    # format whatever is inside the function.
    split_cell = cell.split('$')
    if '-i' not in optdict:
        if len(split_cell) > 1:
            split_cell[0] = split_cell[0].format(**globals())
            split_cell[-1] = split_cell[-1].format(**globals())
            cell = '$'.join(split_cell)
        elif len(split_cell) == 1:
            cell = cell.format(**globals())
    psql.execute(cell, conn)
    refresh_tables(conn)


@register_cell_magic
def printsql(line, cell):
    """Show the SQL query that will be run."""
    
    optlist, args = getopt.getopt(line.split(), 'ih:')
    optdict = dict(optlist)
    
    # Do string formatting. If a PL/Python function
    # is being created, then it should not try and
    # format whatever is inside the function.
    split_cell = cell.split('$')
    if '-i' not in optdict:
        if len(split_cell) > 1:
            split_cell[0] = split_cell[0].format(**globals())
            split_cell[-1] = split_cell[-1].format(**globals())
            cell = '$'.join(split_cell)
        elif len(split_cell) == 1:
            cell = cell.format(**globals())
    print cell


@register_cell_magic
def background(line, cell):
    """Runs whatever is in the cell in a separate thread. This allows
    the user to run cells in the background so that additional cells
    can be run concurrently. This will also micromanage by labelling 
    each thread with an ID number.
    
    Whatever follows after specifying '%%background' will be used as a
    comment to label the process if the ID number is not descriptive
    enough.
    """
    
    def is_useful_code(code):
        """Returns True if code is useful to use. Code that is
        considered not useful is code that contains only commented
        lines or empty lines.
        """

        lines = code.split('\n')

        # For each line, check if line equals '' or starts with a '#'
        return not np.all([line == '' or line[0] == '#' for line in lines])
    
    def run_cell(cell_value, line_value, thread_id, hide_output):
        try:
            exec cell_value in globals()
        except Exception as error_message:
            thread_manager.raise_thread_error(thread_id, error_message)
            raise Exception(error_message)
            
        thread_manager.finish_thread(thread_id, hide_output)
        
    def get_thread_comment(line, sub_comment):
        """Gets the final thread comment by looking at the main comment
        line and the sub-comments.
        """
        
        # Has main comment if the length of the line is greater than 0
        # or if the line is not all spaces
        has_main_comment = len(line) > 0 and not bool(re.match('^( )+$', line))
        
        # Has sub comment if it is not None and if it is not equal to
        # the blank string (The regex will handle cases of all spaces)
        has_sub_comment = sub_comment is not None and sub_comment != ''
        
        if has_main_comment:
            if has_sub_comment:
                return '{} - {}'.format(line, sub_comment)
            else:
                return line
                
        else:
            if has_sub_comment:
                return sub_comment
            else:
                return 'N/A'
        
    optlist, args = getopt.getopt(line.split(), 'h', ['hide'])
    optdict = dict(optlist)
    
    hide_output = '-h' in optdict or '--hide' in optdict
    # Join the arguments to form the line comment
    line_comment = ' '.join(args)
    
    # Splits the code into separate threads by lines that start with
    # two or more '#' characters. 
    cell_list = re.split('(?:^#{2,}|\n#{2,}).*\n', cell)
    cell_list = [sub_cell
                     for sub_cell in cell_list
                         if is_useful_code(sub_cell)]
    
    # Find comment fields, which are defined by lines with two or more
    # '#' characters, spaces, any character (except '#' and '\n'), then
    # a terminating '\n'. Use non-capturing group to ignore '#'
    # characters and spaces before the comment.
    comment_list = re.findall('(?:^#{2,}|\n#{2,})(?: )*([^#\n]*)\n', cell)

    # If there is no break above the first one, comment list will be
    # smaller than cell list, and it will be offset by one.
    if len(comment_list) < len(cell_list):
        comment_list.insert(0, None)
        
    for sub_cell, sub_comment in izip_longest(cell_list, comment_list):
        thread_id = thread_manager.get_next_thread_id()

        # Add thread to thread manager
        thread_comment = get_thread_comment(line_comment, sub_comment)
    
        thread_manager.add_thread(cell_text=sub_cell,
                                  comment=thread_comment,
                                  hide_output=hide_output
                                 )

        # Run the thread in the background
        thread = threading.Thread(target=run_cell,
                                  args=(sub_cell, line_comment, thread_id,
                                        hide_output)
                                 )
        thread.start()

# We delete these to avoid name conflicts for automagic to work
del execsql, readsql, printsql, background